In [7]:
import torch
from torch import nn
import numpy as np

from timeit import default_timer as timer

In [43]:
# create random tensors
torch.manual_seed(42)
y_true = torch.rand(size=(1000,6))
torch.manual_seed(22)
y_pred = torch.rand(size=(1000,6))

y_true[:5], y_pred[:5]

(tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904, 0.6009],
         [0.2566, 0.7936, 0.9408, 0.1332, 0.9346, 0.5936],
         [0.8694, 0.5677, 0.7411, 0.4294, 0.8854, 0.5739],
         [0.2666, 0.6274, 0.2696, 0.4414, 0.2969, 0.8317],
         [0.1053, 0.2695, 0.3588, 0.1994, 0.5472, 0.0062]]),
 tensor([[0.3659, 0.7025, 0.3104, 0.0097, 0.6577, 0.1947],
         [0.9506, 0.6887, 0.8174, 0.7575, 0.7492, 0.6874],
         [0.2564, 0.0672, 0.9066, 0.9175, 0.4236, 0.7978],
         [0.8594, 0.3660, 0.6949, 0.3343, 0.6681, 0.4951],
         [0.3139, 0.7763, 0.7457, 0.5009, 0.4125, 0.1708]]))

In [131]:
y_true = torch.tensor([1, 3, 5]).unsqueeze(dim=1)
y_pred = torch.tensor([1.1, 3.1, 5.1]).unsqueeze(dim=1)

In [26]:
start = timer()

# Calculate element-wise squared differences
squared_diffs = (y_true - y_pred) ** 2
# Compute RMSE for each element
rmse = np.sqrt(squared_diffs)
loss = rmse / (y_true + 1e-20)
output = abs(loss.sum())

end = timer()
time_elapsed = end - start
print(f"time taken = {time_elapsed:.5f} seconds")

time taken = 0.00174 seconds


In [40]:
start = timer()

# Compute RMSE for each element
rmse = torch.nn.functional.mse_loss(y_pred, y_true)
loss = rmse / (y_true + 1e-20)
output = loss.sum().abs()

end = timer()
time_elapsed = end - start
print(f"time taken = {time_elapsed:.5f} seconds")

output

time taken = 0.00085 seconds


tensor(6637.5918)

In [160]:
from sklearn.preprocessing import StandardScaler

y_data_scalar = StandardScaler()
y_pred_scaled = y_data_scalar.fit_transform(y_pred)
y_true_scaled = y_data_scalar.transform(y_true)

y_pred_scaled = torch.tensor(y_pred_scaled, dtype=torch.float32)
y_true_scaled = torch.tensor(y_true_scaled, dtype=torch.float32)

y_pred_scaled[:5], y_true_scaled[:5], y_pred_scaled.dtype

(tensor([[-1.2247e+00],
         [-2.4333e-08],
         [ 1.2247e+00]]),
 tensor([[-1.2860],
         [-0.0612],
         [ 1.1635]]),
 torch.float32)

In [151]:
import numpy as np

def compute_validation_loss(y_true, y_pred, y_data_scalar):

    y_true.numpy()
    y_pred.numpy()

    y_true = y_data_scalar.inverse_transform(y_true)
    y_pred = y_data_scalar.inverse_transform(y_pred)
    # Calculate element-wise squared differences
    squared_diffs = (y_true - y_pred) ** 2

    # Compute RMSE for each element
    rmse = np.sqrt(squared_diffs)

    loss = rmse/(y_true + 1e-20)
    return abs(rmse.sum())

compute_validation_loss(y_true_scaled, y_pred_scaled, y_data_scalar)

0.2999998101560677

In [150]:
from torch import nn

class CustomLossFunction(nn.Module):
    def __init__(self):
        super(CustomLossFunction, self).__init__()
        self.mse = nn.MSELoss(reduction='none')

    def forward(self, y_true, y_pred):
        # Calculate MSE
        mse = self.mse(y_true, y_pred)
        # Calculate RMSE
        rmse = torch.sqrt(mse + 1e-8)

        loss = rmse / (y_true + 1e-20)
        # Calculate custom loss
        # Reduce the loss to a single scalar
        return loss.sum()

LossCalc = CustomLossFunction()

LossCalc(y_true, y_pred)

tensor(0.1533)

In [154]:
from torch import nn

LossCalc = nn.MSELoss(reduction="mean")
LossCalc(y_true, y_pred)

tensor(0.0100)